In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import h5py
import tensorflow as tf
from losses import kl_loss
#import mplhep as hep
#plt.style.use(hep.style.CMS)
#hep.rcParams["yaxis.labellocation"] = 'center'
#hep.rcParams["xaxis.labellocation"] = 'center'
#from losses import threeD_loss, split_3D_loss, mse_split_loss, mse_split_particles_loss, kl_loss

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### Load results

In [ ]:
input_dir = 'VAE_result_qkeras16_1610_half.h5'
data = h5py.File(input_dir, 'r')

In [ ]:
bsm_labels = ['Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']

In [ ]:
# QCD
# for VAE results - QCD = input, QCD_input = target - not anymore (14/06/2021)
X_test_scaled = data['QCD'][:]
X_test = data['QCD_input'][:]
qcd_prediction = data['predicted_QCD'][:]
qcd_mean = data['encoded_mean_QCD'][:]
qcd_logvar = data['encoded_logvar_QCD'][:]
qcd_z = data['encoded_z_QCD'][:]


#BSM
bsm_prediction = []; bsm_data = []; bsm_mean = []; bsm_logvar = []; bsm_z = []; bsm_target=[]
for bsm in bsm_labels[:]:
    bsm_target.append(data[bsm+'_scaled'][:])
    bsm_data.append(data[bsm+'_input'][:])
    bsm_prediction.append(data['predicted_'+ bsm][:])
    bsm_mean.append(data['encoded_mean_'+bsm][:])
    bsm_logvar.append(data['encoded_logvar_'+bsm][:])
    bsm_z.append(data['encoded_z_'+bsm][:])


In [ ]:
# loss = data['loss'][:]
# val_loss = data['val_loss'][:]
# kl_loss = data['kl_loss'][:]
# val_kl_loss = data['val_kl_loss'][:]

In [ ]:
data.close()

## Check distributions of mean and logvar from latent space

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(qcd_mean[:,0], bins=200, histtype='step', density=True, range=(-5,5))
plt.hist(qcd_mean[:,1], bins=200, histtype='step', density=True, range=(-5,5))
plt.hist(qcd_mean[:,2], bins=200, histtype='step', density=True, range=(-5,5))
plt.yscale('log', nonpositive='clip')
plt.ylabel('Prob. Density(a.u.)')
plt.tight_layout()
plt.legend(['qcd mean 1', 'qcd mean 2', 'qcd mean 3'], loc='upper left')
plt.show()

In [ ]:
qcd_sigma = np.sqrt(np.exp(qcd_logvar))
plt.figure(figsize=(10,7))
plt.hist(qcd_logvar[:,0], bins=100, histtype='step', density=True, range=(-20,10))
plt.hist(qcd_logvar[:,1], bins=100, histtype='step', density=True, range=(-20,10))
plt.hist(qcd_logvar[:,2], bins=100, histtype='step', density=True, range=(-20,10))
plt.yscale('log', nonpositive='clip')
plt.ylabel('Prob. Density(a.u.)')
plt.tight_layout()
plt.legend(['qcd logvar 1', 'qcd logvar 2', 'qcd logvar 3'])
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(bsm_mean[0][:,0], bins=200, histtype='step', density=True, range=(-5,5))
plt.hist(bsm_mean[0][:,1], bins=200, histtype='step', density=True, range=(-5,5))
plt.hist(bsm_mean[0][:,2], bins=200, histtype='step', density=True, range=(-5,5))
plt.yscale('log', nonpositive='clip')
plt.ylabel('Prob. Density(a.u.)')
plt.tight_layout()
plt.legend(['LQ mean 1', 'LQ mean 2', 'LQ mean 3'], loc='upper left')
plt.show()

plt.figure(figsize=(10,7))
plt.hist(bsm_mean[1][:,0], bins=200, histtype='step', density=True, range=(-5,5))
plt.hist(bsm_mean[1][:,1], bins=200, histtype='step', density=True, range=(-5,5))
plt.hist(bsm_mean[1][:,2], bins=200, histtype='step', density=True, range=(-5,5))
plt.yscale('log', nonpositive='clip')
plt.ylabel('Prob. Density(a.u.)')
plt.tight_layout()
plt.legend(['Ato4l mean 1', 'Ato4l mean 2', 'Ato4l mean 3'], loc='upper left')
plt.show()

In [ ]:
b_sigma_1 = np.sqrt(np.exp(bsm_logvar[0]))
plt.figure(figsize=(10,7))
plt.hist(bsm_logvar[0][:,0], bins=100, histtype='step', density=True, range=(-50, 5))
plt.hist(bsm_logvar[0][:,1], bins=100, histtype='step', density=True, range=(-50, 5))
plt.hist(bsm_logvar[0][:,2], bins=100, histtype='step', density=True, range=(-50, 5))
plt.yscale('log', nonpositive='clip')
plt.ylabel('Prob. Density(a.u.)')
plt.tight_layout()
plt.legend(['LQ sigma 1', 'LQ sigma 2', 'LQ sigma 3'])
plt.show()

b_sigma_2 = np.sqrt(np.exp(bsm_logvar[1]))
plt.figure(figsize=(10,7))
plt.hist(bsm_logvar[1][:,0], bins=100, histtype='step', density=True, range=(-100, 5))
plt.hist(bsm_logvar[1][:,1], bins=100, histtype='step', density=True, range=(-100, 5))
plt.hist(bsm_logvar[1][:,2], bins=100, histtype='step', density=True, range=(-100, 5))
plt.yscale('log', nonpositive='clip')
plt.ylabel('Prob. Density(a.u.)')
plt.tight_layout()
plt.legend(['Ato4l sigma 1', 'Ato4l sigma 2', 'Ato4l sigma 3'])
plt.show()

### Plot features Test vs Prediction - QCD

In [ ]:
def make_feature_plots(true, prediction, xlabel, particle, bins, density, ranges=None):
    print(find_min_max_range(true, prediction))
    plt.figure(figsize=(7,5))
    if ranges == None: ranges = find_min_max_range(true, prediction) 
    plt.hist(prediction, bins=bins, histtype='step', density=density, range = ranges)
    plt.hist(true, bins=bins, histtype='step', density=density, range = ranges)
    plt.yscale('log', nonpositive='clip')
    plt.ylabel('Prob. Density(a.u.)')
    plt.xlabel(xlabel)
    plt.tight_layout()
    plt.legend([particle+' Predicted', particle+' True'])
    plt.show()

In [ ]:
def make_delta_feature_plots(true, prediction, xlabel, particle, bins, density, ranges=None, phi=False):
    plt.figure(figsize=(7,5))
    if phi:
        delta = (true - prediction)/true
        xlabel = xlabel+' pull'
    else: 
        delta = (true - prediction)/true
        xlabel = xlabel+' pull'
    plt.hist(delta, bins=bins, histtype='step', density=density, range=ranges, label=particle)
    plt.axvline(delta.mean(), color='k', linestyle='dashed', linewidth=1, label='mean = '+str(round(delta.mean(),2)))
    plt.legend(loc='upper right')
    plt.yscale('log', nonpositive='clip')
    plt.ylabel('Prob. Density(a.u.)')
    plt.xlabel(xlabel)
    plt.tight_layout()
    plt.annotate('RMS =  %.2f' % np.sqrt(np.mean(delta**2)), xy=(0, 1), xytext=(12, -12), va='top',\
            xycoords='axes fraction', textcoords='offset points')
    #plt.show()
    
    if 'pT' in xlabel: xlabel = 'pT'
    elif 'phi' in xlabel: xlabel = 'phi'
    elif 'eta' in xlabel: xlabel = 'eta'
    
    plt.savefig(f'plots/{particle}_{xlabel}_zscore_dnn_vae.pdf', facecolor='white')

In [ ]:
def find_min_max_range(true, pred):
    minRange = min(true)
    minPred = min(pred)
    if minPred < minRange: minRange = minPred
        
    maxRange = max(true)
    maxPred = max(pred)
    if maxPred > maxRange: maxRange = maxPred
        
    return (minRange, maxRange)

In [ ]:
#mask_met = X_test[:,0:1]!=0
mask_met_delete = np.where(X_test[:,0:1].reshape(X_test.shape[0]*1)==0)[0]
#mask_eg = X_test[:,1:5]!=0
mask_eg_delete = np.where(X_test[:,1:5].reshape(X_test.shape[0]*4)==0)[0]
#mask_muon = X_test[:,5:9]!=0
mask_muon_delete = np.where(X_test[:,5:9].reshape(X_test.shape[0]*4)==0)[0]
#mask_jet = X_test[:,9:19]!=0
mask_jet_delete = np.where(X_test[:,9:19].reshape(X_test.shape[0]*10)==0)[0]

In [ ]:
# reshape Test and Prediction datasets
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 19, 3, 1)
qcd_pred_reshaped = qcd_prediction.reshape(qcd_prediction.shape[0], 19, 3, 1)

In [ ]:
# MET
make_feature_plots(np.delete(X_test_reshaped[:,0:1,0].reshape(X_test.shape[0]*1),mask_met_delete),\
                   np.delete(qcd_prediction[:,0:1,0].reshape(qcd_prediction.shape[0]*1),mask_met_delete),\
                   'pT', 'MET', 100, True)
make_feature_plots(np.delete(X_test_reshaped[:,0:1,2].reshape(X_test_scaled.shape[0]*1),mask_met_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,0:1,2].reshape(qcd_prediction.shape[0]*1)))),mask_met_delete),\
                   '$\phi$', 'MET', 100, True)
# Jets
make_feature_plots(np.delete(X_test_reshaped[:,9:19,0].reshape(X_test.shape[0]*10),mask_jet_delete),\
                   np.delete(qcd_prediction[:, 9:19,0].reshape(qcd_prediction.shape[0]*10),mask_jet_delete),\
                   'pT', 'Jets', 100, True, ranges=(0,1000))
make_feature_plots(np.delete(X_test_reshaped[:,9:19,1].reshape(X_test.shape[0]*10),mask_jet_delete),\
                np.delete(tf.make_ndarray(tf.make_tensor_proto(4.0*tf.math.tanh(qcd_prediction[:,9:19,1].reshape(qcd_prediction.shape[0]*10)))),mask_jet_delete),\
                   '$\eta$', 'Jets', 100, True)
make_feature_plots(np.delete(X_test_reshaped[:,9:19,2].reshape(X_test.shape[0]*10),mask_jet_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,9:19,2].reshape(qcd_prediction.shape[0]*10)))),mask_jet_delete),\
                   '$\phi$', 'Jets', 100, True) # wrap phi
# Muons
make_feature_plots(np.delete(X_test_reshaped[:,5:9,0].reshape(X_test.shape[0]*4),mask_muon_delete),\
                   np.delete(qcd_prediction[:,5:9,0].reshape(qcd_prediction.shape[0]*4),mask_muon_delete),\
                    'pT', 'Muons', 100, True)
make_feature_plots(np.delete(X_test_reshaped[:,5:9,1].reshape(X_test.shape[0]*4),mask_muon_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(2.1*tf.math.tanh(qcd_prediction[:,5:9,1].reshape(qcd_prediction.shape[0]*4)))),mask_muon_delete),\
                   '$\eta$', 'Muons', 100, True)
make_feature_plots(np.delete(X_test_reshaped[:,5:9,2].reshape(X_test.shape[0]*4),mask_muon_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,5:9,2].reshape(qcd_prediction.shape[0]*4)))),mask_muon_delete),\
                   '$\phi$', 'Muons', 100, True)
#EGammas
make_feature_plots(np.delete(X_test_reshaped[:,1:5,0].reshape(X_test.shape[0]*4),mask_eg_delete),\
                   np.delete(qcd_prediction[:,1:5,0].reshape(qcd_prediction.shape[0]*4),mask_eg_delete),\
                   'pT', 'EGammas', 100, True, ranges = (0.75937235, 500))
make_feature_plots(np.delete(X_test_reshaped[:,1:5,1].reshape(X_test.shape[0]*4),mask_eg_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(3.0*tf.math.tanh(qcd_prediction[:,1:5,1].reshape(qcd_prediction.shape[0]*4)))),mask_eg_delete),\
                   '$\eta$', 'EGammas', 100, True)
make_feature_plots(np.delete(X_test_reshaped[:,1:5,2].reshape(X_test.shape[0]*4),mask_eg_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,1:5,2].reshape(qcd_prediction.shape[0]*4)))),mask_eg_delete),\
                   '$\phi$', 'EGammas', 100, True)


### Plot features (Test - Prediction) - QCD

In [ ]:
# MET
make_delta_feature_plots(np.delete(X_test_reshaped[:,0:1,0].reshape(X_test.shape[0]*1),mask_met_delete),\
                   np.delete(qcd_prediction[:,0:1,0].reshape(qcd_prediction.shape[0]*1),mask_met_delete),\
                   'pT', 'MET', 200, True, ranges=(-1000, 1000))
make_delta_feature_plots(np.delete(X_test_reshaped[:,0:1,2].reshape(X_test.shape[0]*1),mask_met_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,0:1,2].reshape(qcd_prediction.shape[0]*1)))),mask_met_delete),\
                   '$\phi$', 'MET', 200, True, phi=True, ranges=(-200, 200)) # wrap phi
# Jets
make_delta_feature_plots(np.delete(X_test_reshaped[:,9:19,0].reshape(X_test.shape[0]*10),mask_jet_delete),\
                   np.delete(qcd_prediction[:, 9:19,0].reshape(qcd_prediction.shape[0]*10),mask_jet_delete),\
                   'pT', 'Jets', 200, True, ranges=(-10000, 10000))
make_delta_feature_plots(np.delete(X_test_reshaped[:,9:19,1].reshape(X_test.shape[0]*10),mask_jet_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(4.0*tf.math.tanh(qcd_prediction[:,9:19,1].reshape(qcd_prediction.shape[0]*10)))),mask_jet_delete),\
                   '$\eta$', 'Jets', 200, True,phi=True, ranges=(-250,250))
make_delta_feature_plots(np.delete(X_test_reshaped[:,9:19,2].reshape(X_test.shape[0]*10),mask_jet_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,9:19,2].reshape(qcd_prediction.shape[0]*10)))),mask_jet_delete),\
                   '$\phi$', 'Jets', 200, True, phi=True, ranges=(-250, 250)) # wrap phi
# Muons
make_delta_feature_plots(np.delete(X_test_reshaped[:,5:9,0].reshape(X_test.shape[0]*4),mask_muon_delete),\
                   np.delete(qcd_prediction[:,5:9,0].reshape(qcd_prediction.shape[0]*4),mask_muon_delete),\
                    'pT', 'Muons', 200, True, ranges=(-1000,1000))
make_delta_feature_plots(np.delete(X_test_reshaped[:,5:9,1].reshape(X_test.shape[0]*4),mask_muon_delete),\
                 np.delete(tf.make_ndarray(tf.make_tensor_proto(2.1*tf.math.tanh(qcd_prediction[:,5:9,1].reshape(qcd_prediction.shape[0]*4)))),mask_muon_delete),\
                   '$\eta$', 'Muons', 200, True, phi=True, ranges=(-100, 100))
make_delta_feature_plots(np.delete(X_test_reshaped[:,5:9,2].reshape(X_test.shape[0]*4),mask_muon_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,5:9,2].reshape(qcd_prediction.shape[0]*4)))),mask_muon_delete),\
                  '$\phi$', 'Muons', 200, True, phi=True, ranges=(-100, 100))
#EGammas
make_delta_feature_plots(np.delete(X_test_reshaped[:,1:5,0].reshape(X_test.shape[0]*4),mask_eg_delete),\
                   np.delete(qcd_prediction[:,1:5,0].reshape(qcd_prediction.shape[0]*4),mask_eg_delete),\
                   'pT', 'EGammas', 200, True, ranges=(-1000, 1000))
make_delta_feature_plots(np.delete(X_test_reshaped[:,1:5,1].reshape(X_test.shape[0]*4),mask_eg_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(3.0*tf.math.tanh(qcd_prediction[:,1:5,1].reshape(qcd_prediction.shape[0]*4)))),mask_eg_delete),\
                   '$\eta$', 'EGammas', 200, True, phi=True, ranges=(-100, 100))
make_delta_feature_plots(np.delete(X_test_reshaped[:,1:5,2].reshape(X_test.shape[0]*4),mask_eg_delete),\
                   np.delete(tf.make_ndarray(tf.make_tensor_proto(math.pi*tf.math.tanh(qcd_prediction[:,1:5,2].reshape(qcd_prediction.shape[0]*4)))),mask_eg_delete),\
                   '$\phi$', 'EGammas', 200, True, phi=True, ranges=(-100, 100))

### Calculate loss for QCD and BSM data

In [ ]:
from functions import make_mse_loss_numpy, radius

In [ ]:
def return_total_mse_loss(loss, X_test, qcd_prediction, bsm_t, bsm_prediction):
    total_loss = []
    total_loss.append(loss(X_test, qcd_prediction.astype(np.float32)))
    for b, bsm_pred in zip(bsm_t, bsm_prediction):
        total_loss.append(loss(b, bsm_pred.astype(np.float32)))
    return total_loss

def return_total_kl_loss(loss, qcd_mean, qcd_logvar, bsm_mean, bsm_logvar):
    total_loss = []
    total_loss.append(loss(qcd_mean.astype(np.float32), qcd_logvar.astype(np.float32)))
    for mean, logvar in zip(bsm_mean, bsm_logvar):
        total_loss.append(loss(mean.astype(np.float32), logvar.astype(np.float32)))

def return_total_radius(qcd_mean, qcd_logvar, bsm_mean, bsm_logvar):
    total_radius=[]
    total_radius.append(radius(qcd_mean.astype(np.float32), qcd_logvar.astype(np.float32)))
    for mean, logvar in zip(bsm_mean, bsm_logvar):
        total_radius.append(radius(mean.astype(np.float32), logvar.astype(np.float32)))
    return total_radius

In [ ]:
mse_total_loss = return_total_mse_loss(make_mse_loss_numpy, X_test_scaled, qcd_prediction, bsm_target, bsm_prediction)

kl_total_loss = return_total_kl_loss(kl_loss_calc, qcd_mean, qcd_logvar, bsm_mean, bsm_logvar)

total_loss=[]
for mse_loss, kl_loss in zip(mse_total_loss, kl_total_loss):
    print(mse_loss.shape)
    print(kl_loss.shape)
    total_loss.append(np.add(mse_loss, kl_loss))

total_radius = return_total_radius(qcd_mean, qcd_logvar, bsm_mean, bsm_logvar)

### Plot Loss Distributions

In [ ]:
labels = ['QCD multijet', 'Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']

##### Total loss

In [ ]:
#total_loss = tot_loss
minScore = 999999.
maxScore = 0
for i in range(len(labels[:])):
    thisMin = np.min(total_loss[i])
    thisMax = np.max(total_loss[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)

In [ ]:
bin_size=200
plt.figure(figsize=(10,8))
for i, label in enumerate(labels[:]):
    plt.hist(total_loss[i], bins=bin_size, density=True, log=True, label=label, range=(minScore, 1000),
         histtype='step', fill=False, linewidth=1.5)
#plt.semilogx()
plt.semilogy()
#plt.title('MSE')
plt.xlabel("VAE Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.title('(MSE split + KL) loss')
plt.legend(loc='best')
plt.show()

##### MSE (Reconstruction) loss

In [ ]:
minScore = 999999.
maxScore = 0
for i in range(len(labels)):
    thisMin = np.min(mse_total_loss[i])
    thisMax = np.max(mse_total_loss[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)
for i in range(len(labels)):
    thisMin = np.min(kl_total_loss[i])
    thisMax = np.max(kl_total_loss[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)

In [ ]:
bin_size=200;colors=['C1', 'C2', 'C3', 'C4', 'C5']
plt.figure(figsize=(15,13))
for i, label in enumerate(labels):
    plt.hist(mse_total_loss[i], bins=bin_size, density=True, log=True, label=label+' MSE', range=(minScore, 50),
         histtype='step', fill=False, linewidth=1.5, color=colors[i])
    plt.hist(kl_total_loss[i], bins=bin_size, label=label+' KL', density = True, log=True, range=(minScore, 50),
         histtype='step', fill=False, linewidth=1.5, color=colors[i], alpha=0.5)
#plt.semilogx()
plt.semilogy()
#plt.title('MSE')
plt.xlabel("VAE Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
#plt.title('Not quantized model')
plt.legend(loc='best')
plt.show()

##### KL divergence loss

In [ ]:
minScore = 999999.
maxScore = 0
for i in range(len(labels)):
    thisMin = np.min(kl_total_loss[i])
    thisMax = np.max(kl_total_loss[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)

In [ ]:
bin_size=200
plt.figure(figsize=(10,8))
for i, label in enumerate(labels):
    plt.hist(kl_total_loss[i].reshape(kl_total_loss[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, 1),
         histtype='step', fill=False, linewidth=1.5)
#plt.semilogx()
plt.semilogy()
plt.xlabel("Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.title('KL loss')
plt.legend(loc='best')
plt.show()

##### Radius loss

In [ ]:
tot_radius = []
for radius in total_radius:
    r = np.nan_to_num(radius)
    r[r==-np.inf] = 0
    r[r==np.inf] = 0
    r[r>=1E308] = 0
    tot_radius.append(r)
del total_radius

In [ ]:
minS = 999999.
maxS = 0.
for i in range(len(tot_radius)):
    thisMin = np.min(tot_radius[i])
    thisMax = np.max(tot_radius[i])
    minS = min(thisMin, minS)
    maxS = max(maxS, thisMax)

In [ ]:
plt.figure(figsize=(10,8))
for i, label in enumerate(labels):
    plt.hist(tot_radius[i], bins=200, label=label, density=True, range=(minS, 1000),
             histtype='step', fill=False, linewidth=1.5)
plt.semilogy()
plt.title(' ')
plt.xlabel("$\mu^2 / \sigma^2$")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

### Plot ROC curves

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
colors = ['C1','C2', 'C3', 'C4']
#colors   = ['#a6bddb','#67a9cf','#3690c0','#02818a','#016c59','#014636']

In [ ]:
tpr_lq=[];fpr_lq=[];auc_lq=[]
tpr_ato4l=[];fpr_ato4l=[];auc_ato4l=[]
tpr_ch=[];fpr_ch=[];auc_ch=[]
tpr_to=[];fpr_to=[];auc_to=[]

###### tpr, fpr, auc for total loss

In [ ]:
target_qcd = np.zeros(total_loss[0].shape[0])

for i, label in enumerate(labels):
    if i == 0: continue
    trueVal = np.concatenate((np.ones(total_loss[i].shape[0]), target_qcd))
    predVal_loss = np.concatenate((total_loss[i], total_loss[0]))
    fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)
    auc_loss = auc(fpr_loss, tpr_loss)
    if i==1:
        tpr_lq.append(tpr_loss)
        fpr_lq.append(fpr_loss)
        auc_lq.append(auc_loss)
    elif i == 2:
        tpr_ato4l.append(tpr_loss)
        fpr_ato4l.append(fpr_loss)
        auc_ato4l.append(auc_loss)
    elif i==3:
        tpr_ch.append(tpr_loss)
        fpr_ch.append(fpr_loss)
        auc_ch.append(auc_loss)
    elif i == 4:
        tpr_to.append(tpr_loss)
        fpr_to.append(fpr_loss)
        auc_to.append(auc_loss)

###### tpr, fpr, auc for MSE loss

In [ ]:
target_qcd = np.zeros(mse_total_loss[0].shape[0])

for i, label in enumerate(labels):
    if i == 0: continue
    trueVal = np.concatenate((np.ones(mse_total_loss[i].shape[0]), target_qcd))
    predVal_loss = np.concatenate((mse_total_loss[i], mse_total_loss[0]))

    fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

    auc_loss = auc(fpr_loss, tpr_loss)
    if i==1:
        tpr_lq.append(tpr_loss)
        fpr_lq.append(fpr_loss)
        auc_lq.append(auc_loss)
    elif i == 2:
        tpr_ato4l.append(tpr_loss)
        fpr_ato4l.append(fpr_loss)
        auc_ato4l.append(auc_loss)
    elif i==3:
        tpr_ch.append(tpr_loss)
        fpr_ch.append(fpr_loss)
        auc_ch.append(auc_loss)
    elif i == 4:
        tpr_to.append(tpr_loss)
        fpr_to.append(fpr_loss)
        auc_to.append(auc_loss)

###### tpr, fpr, auc for KL divergence loss

In [ ]:
target_qcd = np.zeros(kl_total_loss[0].shape[0])
#target_qcd_rand = np.zeros(kl_total_loss[3].shape[0])

for i, label in enumerate(labels):
    if i == 0: continue
    trueVal = np.concatenate((np.ones(kl_total_loss[i].shape[0]), target_qcd))
    predVal_loss = np.concatenate((kl_total_loss[i], kl_total_loss[0]))

    fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

    auc_loss = auc(fpr_loss, tpr_loss)
    if i==1:
        tpr_lq.append(tpr_loss)
        fpr_lq.append(fpr_loss)
        auc_lq.append(auc_loss)
    elif i == 2:
        tpr_ato4l.append(tpr_loss)
        fpr_ato4l.append(fpr_loss)
        auc_ato4l.append(auc_loss)
    elif i==3:
        tpr_ch.append(tpr_loss)
        fpr_ch.append(fpr_loss)
        auc_ch.append(auc_loss)
    elif i == 4:
        tpr_to.append(tpr_loss)
        fpr_to.append(fpr_loss)
        auc_to.append(auc_loss)

###### tpr, fpr, auc for Radius loss

In [ ]:
target_qcd = np.zeros(tot_radius[0].shape[0])
#target_qcd_rand = np.zeros(tot_radius[3].shape[0])

for i, label in enumerate(labels):
    if i == 0: continue
    trueVal = np.concatenate((np.ones(tot_radius[i].shape[0]), target_qcd))
    predVal_loss = np.concatenate((tot_radius[i], tot_radius[0]))

    fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

    auc_loss = auc(fpr_loss, tpr_loss)
    if i==1:
        tpr_lq.append(tpr_loss)
        fpr_lq.append(fpr_loss)
        auc_lq.append(auc_loss)
    elif i == 2:
        tpr_ato4l.append(tpr_loss)
        fpr_ato4l.append(fpr_loss)
        auc_ato4l.append(auc_loss)
    elif i==3:
        tpr_ch.append(tpr_loss)
        fpr_ch.append(fpr_loss)
        auc_ch.append(auc_loss)
    elif i == 4:
        tpr_to.append(tpr_loss)
        fpr_to.append(fpr_loss)
        auc_to.append(auc_loss)

In [ ]:
losses = ['Total loss', 'MSE VAE ', '$D_{KL}$', '$R_{z}$']

In [ ]:
labels = [r'LQ $\rightarrow$ b$\tau$', r'A $\rightarrow$ 4L', r'$h_{\pm} \rightarrow \tau\nu$', r'$h_{0} \rightarrow \tau\tau$']

### ROCs for LQ signal

In [ ]:
plt.figure(figsize=(10,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_lq[1:], fpr_lq[1:], auc_lq[1:], losses[1:])):
    if i >= 4:
        plt.plot(fpr, tpr, "-", linewidth=1.5, color=colors[i-4], alpha=0.4)
    else:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L ,auc*100.), linewidth=1.5, color=colors[i])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend(loc='lower right',frameon=True, fontsize=20)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
plt.title("ROC "+labels[0]+" Baseline Pruned")
#plt.savefig('ROC_LQ.pdf', facecolor='white')
plt.show()

### ROCs for A to 4 leptons signal

In [ ]:
plt.figure(figsize=(10,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ato4l[1:], fpr_ato4l[1:], auc_ato4l[1:], losses[1:])):
    if i >= 4:
        plt.plot(fpr, tpr, "-",linewidth=1.5, color=colors[i-4],alpha=0.4)
    else:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L ,auc*100.), linewidth=1.5, color=colors[i])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend(loc='lower right',frameon=True, fontsize=20)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
plt.title("ROC "+labels[1]+" Baseline Pruned")
#plt.savefig('ROC_ato4l.pdf', facecolor='white')
plt.show()

### ROCs for higgs charged signal

In [ ]:
plt.figure(figsize=(10,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ch[1:], fpr_ch[1:], auc_ch[1:], losses[1:])):
    if i >= 4:
        plt.plot(fpr, tpr, "-", linewidth=1.5, color=colors[i-4],alpha=0.5)
    else:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L ,auc*100.), linewidth=1.5, color=colors[i])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend(loc='lower right',frameon=True, fontsize=20)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
plt.title("ROC "+labels[2]+" Baseline Pruned")
#plt.savefig('ROC_CH.pdf', facecolor='white')
plt.show()

### ROCs for higgs signal

In [ ]:
plt.figure(figsize=(10,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_to[1:], fpr_to[1:], auc_to[1:], losses[1:])):
    if i >= 4:
        plt.plot(fpr, tpr, "-", linewidth=1.5, color=colors[i-4],alpha=0.5)
    else:
        plt.plot(fpr, tpr, "-", label='%s ROC (auc = %.1f%%)'%(L ,auc*100.), linewidth=1.5, color=colors[i])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend(loc='lower right',frameon=True, fontsize=20)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
plt.title("ROC "+labels[3]+" Baseline Pruned")
plt.savefig('ROC_TO.pdf', facecolor='white')
plt.show()
